<a href="https://colab.research.google.com/github/EldarAlimov/Pet-Projects/blob/main/CropClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image

In [ ]:
import zipfile
import os

train_zip = '/content/drive/My Drive/CropDatasets/train.zip'
test_zip = '/content/drive/My Drive/CropDatasets/test.zip'

extract_dir = '/content/dataset'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(extract_dir, 'train'))

with zipfile.ZipFile(test_zip, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(extract_dir, 'test'))

In [ ]:
train_dir = '/content/dataset/train/train'
test_dir = '/content/dataset/test'

print("Train directory contents:")
print(os.listdir(train_dir))

print("\nTest directory contents:")
print(os.listdir(test_dir))

Train directory contents:
['images', 'train.csv', '.DS_Store']

Test directory contents:
['__MACOSX', 'test']


In [ ]:
class CropDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, class_to_idx=None, idx_to_class=None):
        self.data = pd.read_csv(csv_file)


        self.data = self.data.dropna(subset=['class'])
        valid_classes = set(class_to_idx.keys())
        self.data = self.data[self.data['class'].isin(valid_classes)]

        self.root_dir = root_dir
        self.transform = transform
        self.class_to_idx = class_to_idx
        self.idx_to_class = idx_to_class

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        label = self.data.iloc[idx, 1]
        if self.class_to_idx:
            label = self.class_to_idx[label]
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
train_csv = '/content/dataset/train/train/train.csv'
train_images ='/content/dataset/train/train/images'
test_csv = '/content/dataset/test/test/test.csv'
test_images ='/content/dataset/test/test/images'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class_to_idx = {'flax': 0, 'wheat': 1, 'oats': 2, 'barley': 3}
idx_to_class = {0:'flax', 1: 'wheat', 2:'oats', 3:'barley'}


In [ ]:
train_dataset = CropDataset(
    csv_file=train_csv,
    root_dir=train_images,
    transform=transform,
    class_to_idx=class_to_idx,
    idx_to_class=idx_to_class
)

test_dataset = CropDataset(
    csv_file=test_csv,
    root_dir=test_images,
    transform=transform,
    class_to_idx=class_to_idx,
    idx_to_class=idx_to_class
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#train_dataset.data['class'] = train_dataset.data['class'].map(class_to_idx)
#test_dataset.data['class'] = test_dataset.data['class'].map(class_to_idx)


print(train_dataset.data.head())


                                            img   class
0  niwlxswqmehvazrydmoghchajsvmjmkhypzkyqga.jpg   wheat
1  ivkvufzmpjagkwtcaimkzaeuuvgpsnggjrexgbzh.jpg  barley
2  qczfljfahddkmpzcbxnhvrouritxrltypgvllbzh.jpg   wheat
3  tpctjrjupgejtkbmqerqxxpxdjfzylmczevppnla.jpg   wheat
4  pdoxmibknwaldsrnnuewdvnatkxkcpemnpolfcwy.jpg    flax


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)

num_classes = 4
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_epochs = 30
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")


<ipython-input-14-ee47be2476d6>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch [1/30], Loss: 58.4467
Epoch [2/30], Loss: 33.8667
Epoch [3/30], Loss: 25.6772
Epoch [4/30], Loss: 18.5113
Epoch [5/30], Loss: 13.1414
Epoch [6/30], Loss: 11.3647
Epoch [7/30], Loss: 10.6258
Epoch [8/30], Loss: 9.5556
Epoch [9/30], Loss: 7.7430
Epoch [10/30], Loss: 7.0042
Epoch [11/30], Loss: 3.3002
Epoch [12/30], Loss: 7.1699
Epoch [13/30], Loss: 5.2127
Epoch [14/30], Loss: 15.3463
Epoch [15/30], Loss: 4.0050
Epoch [16/30], Loss: 2.6722
Epoch [17/30], Loss: 0.7573
Epoch [18/30], Loss: 6.2748
Epoch [19/30], Loss: 5.1692
Epoch [20/30], Loss: 2.2986
Epoch [21/30], Loss: 1.5070
Epoch [22/30], Loss: 1.2201
Epoch [23/30], Loss: 1.8476
Epoch [24/30], Loss: 3.6670
Epoch [25/30], Loss: 10.5746
Epoch [26/30], Loss: 5.8788
Epoch [27/30], Loss: 1.5920
Epoch [28/30], Loss: 1.3679
Epoch [29/30], Loss: 2.1473
Epoch [30/30], Loss: 1.8251


In [ ]:
save_path = '/content/drive/My Drive/SharedFolder/model/resnet18.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"Model saved at {save_path}")

Model saved at /content/drive/My Drive/SharedFolder/model/resnet18.pth


In [ ]:
model.eval()
correct = 0
total = 0

results = []

with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)


        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for i, prediction in enumerate(predicted):
            img_name = test_dataset.data.iloc[i, 0]
            predicted_class = idx_to_class[prediction.item()]
            results.append([img_name, predicted_class])

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 0.00%
